In [1]:
#%%     
# PLOTTING WITH GEOID, WESTCHESTER SHP NOT FOUND YET, NEED TO BLACK OUT BRROKLYN AND STATEN ISLAND
# Plotting the census tracts for all reports that appeared in a specific month                    
# import contextily as ctx
# import matploylib.pyplot as plt
import geopandas as gp
import os
import platform
import pandas as pd
import numpy as np
import json
# A) CREATING DF AND GDF:
csvFile     = "DataFiles/ConEdison/GasHistory_2010_ReportFrequency_Monthly.csv"
outNYCFile  = "DataFiles/ConEdison/GasHistory_2010_ReportFrequency_Monthly_nycPlot.csv"
outWestFile = "DataFiles/ConEdison/GasHistory_2010_ReportFrequency_Monthly_westchesterPlot.csv"

nycFile         = "ShapeFiles/NYU_NYC_34505_SP/nyu_2451_34505.shp"
brooklynFile    = "ShapeFiles/TIGER_2010_County_Shapefiles/BrooklynCounty_2010SHP/tl_2010_36047_tract10.shp"
statenFile      = "ShapeFiles/TIGER_2010_County_Shapefiles/StatenIslanCounty_2010SHP"
westchesterFile = "ShapeFiles/westchester2010/wctigtrt10.shp"

geoGDFFile2     = "ShapeFiles/OpenNYC_NYC_JeoSON/nyu_2451_34510_2/nyu_2451_34510.shp"#"ShapeFiles/OpenNYC_NYC_JeoSON/2010 Census Tracts/geo_export_4c66e290-408d-4606-b952-01ee89f0fd2e.shp"
geoGDFFile1     =  "ShapeFiles/OpenNYC_NYC_JeoSON/2010 Census Tracts/geo_export_4c66e290-408d-4606-b952-01ee89f0fd2e.shp"
geoJsonNYCFile = "ShapeFiles/OpenNYC_NYC_JeoSON/2010 Census Tracts.geojson"
# import pygeoj
# testfile = pygeoj.load(filepath=geoJsonNYCFile)

geoGDFFile3 = "ShapeFiles/OpenNYC_NYC_JeoSON/2010 Census Tracts 2/geo_export_a9a9fd7a-e259-403a-bda5-a44d36a58b20.shp"
# https://data.cityofnewyork.us/City-Government/2010-Census-Tracts/fxpq-c8ku

In [2]:
monthlyDF = pd.read_csv(csvFile)                                                                            # Read the csv file and make a data frame
nycGDF = gp.read_file(nycFile)                                                                           # Read the shape file and make a data frame
brooklynGDF = gp.read_file(brooklynFile)
statenGDF = gp.read_file(statenFile)
westchesterGDF = gp.read_file(westchesterFile)
print(type(nycGDF))
print(type(nycGDF["geometry"]))
print(type(nycGDF["geometry"][0]))

geoGDF1 = gp.read_file(geoGDFFile1)
geoGDF2 = gp.read_file(geoGDFFile2)
geoGDF3 = gp.read_file(geoGDFFile3) #they made a graph soehow
geoGDF1 # THIS IS MY COORDINTE RN

<class 'geopandas.geodataframe.GeoDataFrame'>
<class 'geopandas.geoseries.GeoSeries'>
<class 'shapely.geometry.polygon.Polygon'>


,boro_code,boro_ct201,boro_name,cdeligibil,ct2010,ctlabel,ntacode,ntaname,puma,shape_area,shape_leng,geometry
0,5,5000900,Staten Island,E,000900,9,SI22,West New Brighton-New Brighton-St. George,3903,2.497010e+06,7729.016794,POLYGON ((-74.07920577013245 40.64343078374567...
1,1,1009800,Manhattan,I,009800,98,MN19,Turtle Bay-East Midtown,3808,1.906016e+06,5534.199811,POLYGON ((-73.96432543478758 40.75638153099091...
2,1,1010000,Manhattan,I,010000,100,MN19,Turtle Bay-East Midtown,3808,1.860938e+06,5692.168737,POLYGON ((-73.96802436915851 40.75957814005282...
3,1,1010200,Manhattan,I,010200,102,MN17,Midtown-Midtown South,3807,1.860993e+06,5687.802439,POLYGON ((-73.97124277307127 40.76093641847906...
4,1,1010400,Manhattan,I,010400,104,MN17,Midtown-Midtown South,3807,1.864600e+06,5693.036367,POLYGON ((-73.97445730550224 40.76229308352487...
...,...,...,...,...,...,...,...,...,...,...,...,...
2160,4,4020500,Queens,I,020500,205,QN31,Hunters Point-Sunnyside-West Maspeth,4109,6.332771e+06,11121.116286,POLYGON ((-73.91796696706498 40.72850077062928...
2161,4,4019900,Queens,I,019900,199,QN31,Hunters Point-Sunnyside-West Maspeth,4109,1.713161e+07,21121.418668,POLYGON ((-73.93022653420849 40.74244577165481...
2162,4,4021900,Queens,I,021900,219,QN31,Hunters Point-Sunnyside-West Maspeth,4109,1.374685e+07,22381.537560,"POLYGON ((-73.90712253078468 40.7274247284378,..."
2163,2,2011900,Bronx,E,011900,119,BX27,Hunts Point,3710,1.804162e+06,5577.769510,"POLYGON ((-73.8875593465937 40.82247356009054,..."


In [3]:
geoGDF2 # WANT THE PLY TO HAVE THIS COORDINATES

,statefp,countyfp,countyns,geoid,name,namelsad,lsad,classfp,mtfcc,csafp,cbsafp,metdivfp,funcstat,aland,awater,intptlat,intptlon,geometry
0,36,005,00974101,36005,Bronx,Bronx County,06,H6,G4020,408,35620,35644,C,109028913,39838232,+40.8487111,-073.8529390,"POLYGON ((1002596.675078722 260789.8743896104,..."
1,36,047,00974122,36047,Kings,Kings County,06,H6,G4020,408,35620,35644,C,183398743,67615902,+40.6351332,-073.9507774,"POLYGON ((969093.7823651452 177334.5333386248,..."
2,36,061,00974129,36061,New York,New York County,06,H6,G4020,408,35620,35644,C,59144826,27854541,+40.7765574,-073.9701736,"(POLYGON ((974697.4522795392 189507.905372635,..."
3,36,081,00974139,36081,Queens,Queens County,06,H6,G4020,408,35620,35644,C,281177201,180405065,+40.6585571,-073.8379293,"POLYGON ((985059.4481387105 142309.7274380878,..."
4,36,085,00974141,36085,Richmond,Richmond County,06,H6,G4020,408,35620,35644,C,151184452,113749023,+40.5638553,-074.1370633,"POLYGON ((914970.4140434263 137904.8788970875,..."


In [4]:
geoGDF3

,boro_code,boro_ct201,boro_name,cdeligibil,ct2010,ctlabel,ntacode,ntaname,puma,shape_area,shape_leng,geometry
0,5,5000900,Staten Island,E,000900,9,SI22,West New Brighton-New Brighton-St. George,3903,2.497010e+06,7729.016794,POLYGON ((-74.07920577013245 40.64343078374567...
1,1,1009800,Manhattan,I,009800,98,MN19,Turtle Bay-East Midtown,3808,1.906016e+06,5534.199811,POLYGON ((-73.96432543478758 40.75638153099091...
2,1,1010000,Manhattan,I,010000,100,MN19,Turtle Bay-East Midtown,3808,1.860938e+06,5692.168737,POLYGON ((-73.96802436915851 40.75957814005282...
3,1,1010200,Manhattan,I,010200,102,MN17,Midtown-Midtown South,3807,1.860993e+06,5687.802439,POLYGON ((-73.97124277307127 40.76093641847906...
4,1,1010400,Manhattan,I,010400,104,MN17,Midtown-Midtown South,3807,1.864600e+06,5693.036367,POLYGON ((-73.97445730550224 40.76229308352487...
...,...,...,...,...,...,...,...,...,...,...,...,...
2160,4,4020500,Queens,I,020500,205,QN31,Hunters Point-Sunnyside-West Maspeth,4109,6.332771e+06,11121.116286,POLYGON ((-73.91796696706498 40.72850077062928...
2161,4,4019900,Queens,I,019900,199,QN31,Hunters Point-Sunnyside-West Maspeth,4109,1.713161e+07,21121.418668,POLYGON ((-73.93022653420849 40.74244577165481...
2162,4,4021900,Queens,I,021900,219,QN31,Hunters Point-Sunnyside-West Maspeth,4109,1.374685e+07,22381.537560,"POLYGON ((-73.90712253078468 40.7274247284378,..."
2163,2,2011900,Bronx,E,011900,119,BX27,Hunts Point,3710,1.804162e+06,5577.769510,"POLYGON ((-73.8875593465937 40.82247356009054,..."


In [5]:
#now mine
# import json 
  
# # def returnGeoJSON_fromGDF(gdf):
# #     jsonStr = json.loads(gdf.to_json())
# #     json_data = json.dumps(jsonStr)
# #     jsonDict = json.loads(json_data) 
# #     return jsonDict
# # print(returnGeoJSON_fromGDF(geoGDF))
# j = json.stringify(geoJsonNYCFile)


In [6]:
print(geoGDF.iloc[0]["geometry"])

NameError: name 'geoGDF' is not defined

In [ ]:
geoGDF


In [ ]:
print(nycGDF.iloc[0]["geometry"])

In [ ]:
figx = 14
figy = 13
map = geoGDF.plot(column='boro_name',cmap = 'Reds', edgecolor='black', linewidth = 0.3, figsize = (figx,figy),legend = True)#, ax=ax, alpha=1) #10,8
